In [ ]:
#for Checking randomly generated ip addreeses from the csv file-->>
    #random_ip_addresses = random.sample(ip_tokens, min(100, len(ip_tokens)))
    #results = check_ip_addresses_with_abuseipdb(random_ip_addresses)

In [ ]:
import os
import re
import random
import requests
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import PatternFill
import shutil


# Define your AbuseIPDB API key and base URL
API_KEY = '# enter your personal API key '
BASE_URL = 'https://api.abuseipdb.com/api/v2/check'

# Extract IP addresses from text
def extract_ip_addresses(text):
    ip_pattern = r'\b(?:\d{1,3}\.){3}\d{1,3}\b'
    return re.findall(ip_pattern, text)

# Read all CSV files from a folder and tokenize IP addresses
def tokenize_csv_and_select_ips(folder_path):
    ip_tokens = []

    for csv_file in os.listdir(folder_path):
        if csv_file.endswith('.csv'):
            csv_file_path = os.path.join(folder_path, csv_file)
            try:
                df = pd.read_csv(csv_file_path,encoding='latin-1')
                for _, row in df.iterrows():
                    ip_addresses = extract_ip_addresses(row['IP'])  # Assuming 'IP' is the column name
                    ip_tokens.extend(ip_addresses)
            except pd.errors.EmptyDataError:
                print(f"CSV file '{csv_file}' is empty.")
    
    return ip_tokens

# Check IP addresses against AbuseIPDB API
def check_ip_addresses_with_abuseipdb(ip_addresses):
    results = []

    for ip_address in ip_addresses:
        params = {
            'ipAddress': ip_address,
            'maxAgeInDays': 90,  # Adjust as needed
            'verbose': True,
        }
        headers = {
            'Key': API_KEY,
        }

        response = requests.get(BASE_URL, params=params, headers=headers)
        if response.status_code == 200:
            result = response.json()
            results.append(result)
        else:
            print(f"Error checking IP address {ip_address}: {response.status_code}")

    return results

# Apply fill color to Excel cell based on abuse confidence score
def apply_fill_color(cell, abuse_score):
    if 0 <= abuse_score <= 30:
        cell.fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")  # Green
    elif 31 <= abuse_score <= 70:
        cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")  # Yellow
    elif 71 <= abuse_score <= 100:
        cell.fill = PatternFill(start_color="FFA500", end_color="FFA500", fill_type="solid")  # Amber

# Save AbuseIPDB results to Excel
def save_results_to_excel(results):
    excel_folder = ' # desired output folder'  
    excel_filename = os.path.join(excel_folder, 'abuse_ip_report.xlsx')

    wb = Workbook()
    ws = wb.active
    ws.title = 'Abuse IP Report'

    column_headers = ['IP Address', 'Abuse Confidence Score', 'Total Reports', 'Is Public', 'Is Whitelisted', 'Country Code']
    ws.append(column_headers)

    for result in results:
        data = [
            result['data']['ipAddress'],
            result['data']['abuseConfidenceScore'],
            result['data']['totalReports'],
            result['data']['isPublic'],
            result['data']['isWhitelisted'],
            result['data']['countryCode']
        ]
        ws.append(data)
        cell = ws.cell(row=ws.max_row, column=2)  # Column 2 contains 'Abuse Confidence Score'
        abuse_score = result['data']['abuseConfidenceScore']
        apply_fill_color(cell, abuse_score)

    wb.save(excel_filename)
    print(f"Results saved to {excel_filename}")

# Organize files
def organize_files(folder_path):
    csv_folder_path = os.path.join(folder_path, "only_csv_files")
    os.makedirs(csv_folder_path, exist_ok=True)

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            _, ext = os.path.splitext(filename)
            if ext.lower() == ".csv":
                
                # Move CSV files to the 'only_csv_files' folder
                new_file_path = os.path.join(csv_folder_path, filename)
                shutil.move(file_path, new_file_path)
            else:
                # Create a copy with .csv extension and move to 'only_csv_files' folder
                new_filename = os.path.splitext(filename)[0] + ".csv"
                new_file_path = os.path.join(csv_folder_path, new_filename)
                shutil.copy(file_path, new_file_path)
    
    print("Organizing files completed.")


def main():
    folder_path = '# Give Path for folder with all files related to IOC check'
    
    # Organize files
    organize_files(folder_path)
    
    # Tokenize IP addresses from all CSV files
    ip_tokens = tokenize_csv_and_select_ips(os.path.join(folder_path, "only_csv_files"))
    
    if not ip_tokens:
        print("No IP addresses found in the CSV files.")
        return

    random_ip_addresses = random.sample(ip_tokens, min(100, len(ip_tokens)))
    results = check_ip_addresses_with_abuseipdb(random_ip_addresses)

    save_results_to_excel(results)

if __name__ == "__main__":
    main()
